# Environement Setup

In [1]:
# @title 1. 🛠️ Installazione Ambiente (PyTorch 2.4.0)
import os
import sys

print("🔄 Preparazione ambiente in corso... (richiederà 2-3 minuti)")

# 1. Disinstallazione versioni incompatibili
os.system("pip uninstall -y torch torchvision torchaudio torch-scatter torch-sparse torch-cluster torch-geometric")

# 2. Installazione PyTorch 2.4.0 (Versione richiesta dal README)
# Usiamo CUDA 12.1 che è stabile su Colab
cmd_torch = "pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121"
os.system(cmd_torch)

# 3. Installazione librerie Grafiche (PyG) specifiche per Torch 2.4.0
pyg_url = "https://data.pyg.org/whl/torch-2.4.0+cu121.html"
libs = ["torch-scatter", "torch-sparse", "torch-cluster", "torch-spline-conv", "torch-kmeans"]
cmd_pyg = f"pip install {' '.join(libs)} -f {pyg_url}"
os.system(cmd_pyg)

# 4. Altre dipendenze
os.system("pip install torch-geometric hydra-core omegaconf submitit pandas ftfy")

os.system("pip install -r requirements.txt")

print("✅ Installazione completata.")
print("⚠️ IMPORTANTE: Ora vai nel menu in alto: 'Runtime' > 'Restart Session' (Riavvia Sessione) prima di eseguire la cella successiva!")

🔄 Preparazione ambiente in corso... (richiederà 2-3 minuti)
✅ Installazione completata.
⚠️ IMPORTANTE: Ora vai nel menu in alto: 'Runtime' > 'Restart Session' (Riavvia Sessione) prima di eseguire la cella successiva!


In [1]:
# @title 2. 📂 Setup HiERO e Download Pesi Pre-addestrati
import os
from google.colab import userdata

# 1. Recupero Token e Clone del repository Privato
repo_name = "HiERO-main"
if not os.path.exists(repo_name):
    print(f"⬇️ Clonazione repository {repo_name} (Privato)...")
    try:
        # Recupera il token dai segreti di Colab
        token = userdata.get('GITHUB_TOKEN')

        # Costruisce l'URL autenticato: https://TOKEN@github.com/...
        repo_url = f"https://{token}@github.com/MarcoPernoVDP/HiERO-main.git"

        # Esegue il clone (l'output viene nascosto per sicurezza se usi capture_output,
        # ma os.system standard è ok, basta non stampare l'URL completo)
        ret = os.system(f"git clone --recursive {repo_url}")

        if ret != 0:
            print("❌ Errore durante il clone. Verifica che il token sia corretto e abbia i permessi.")
    except Exception as e:
        print(f"❌ Errore nel recupero del token: {e}")
        print("Assicurati di aver aggiunto 'GITHUB_TOKEN' nei Segreti di Colab (icona chiave a sinistra).")
else:
    print(f"✅ Repository {repo_name} già presente.")

# Cambiamo directory se il clone ha avuto successo o la cartella esisteva
if os.path.exists(f"/content/{repo_name}"):
    os.chdir(f"/content/{repo_name}")
    print(f"📂 Directory di lavoro: {os.getcwd()}")

    # 2. Creazione struttura cartelle
    dirs = [
        "checkpoints",
        "pretrained",
        "data/ego4d/raw/annotations",
        "data/ego4d/raw/features/egovlp"
    ]
    for d in dirs:
        os.makedirs(d, exist_ok=True)

    # 3. Download del modello HiERO già addestrato (Backbone EgoVLP)
    if not os.path.exists("checkpoints/hiero_egovlp.pth"):
        print("⬇️ Scaricamento Modello Pre-addestrato (hiero_egovlp.pth)...")
        os.system("wget -O checkpoints/hiero_egovlp.pth https://www.sapeirone.it/data/hiero/ckpt/hiero_egovlp.pth")

    # 4. Download componenti testuali (richiesti dal codice)
    print("⬇️ Scaricamento Encoder Testuale...")
    os.system("wget -q -O pretrained/egovlp_text.pth https://www.sapeirone.it/data/hiero/pretrained/egovlp_text.pth")
    os.system("wget -q -O pretrained/egovlp_txt_proj.pth https://www.sapeirone.it/data/hiero/pretrained/egovlp_txt_proj.pth")

    # 5. Download Annotazioni necessarie (EgoMCQ/EgoClip)
    print("⬇️ Scaricamento Annotazioni...")
    os.system("gdown --id 1-aaDu_Gi-Y2sQI_2rsI2D1zvQBJnHpXl -O data/ego4d/raw/annotations/egoclip.csv")
    os.system("gdown --id 1-5iRYf4BCHmj4MYQYFRMY4bhsWJUN3rW -O data/ego4d/raw/annotations/egomcq.json")

    print("✅ Setup completato. Cartelle pronte.")
else:
    print("⚠️ Impossibile procedere: la cartella del repository non esiste.")

⬇️ Clonazione repository HiERO-main (Privato)...
📂 Directory di lavoro: /content/HiERO-main
⬇️ Scaricamento Modello Pre-addestrato (hiero_egovlp.pth)...
⬇️ Scaricamento Encoder Testuale...
⬇️ Scaricamento Annotazioni...
✅ Setup completato. Cartelle pronte.


In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
import zipfile

zip_path = "/content/drive/MyDrive/MistakeDetection/egovlp.zip"
extract_path = "/content/data/egovlp"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

Mounted at /content/drive


In [ ]:
%cd /content/HiERO-main

!python batch_infer_text.py \
  --recipes-dir /content/drive/MyDrive/MistakeDetection/task_graphs \
  --checkpoint /content/HiERO-main/checkpoints/hiero_egovlp.pth \
  --output-dir /content/drive/MyDrive/MistakeDetection/recipe_text_step_embeddings \
  --format pt \
  --device cuda

/content/HiERO-main
🍳 Trovati 24 file JSON da processare
📁 Output directory: /content/drive/MyDrive/MistakeDetection/recipe_text_step_embeddings
💾 Formato: pt
🖥️  Device: cuda

[1/24] 🔄 Processing: herbomeletwithfriedtomatoes
✅ Completato: herbomeletwithfriedtomatoes

[2/24] 🔄 Processing: mugcake
✅ Completato: mugcake

[3/24] 🔄 Processing: capresebruschetta
✅ Completato: capresebruschetta

[4/24] 🔄 Processing: spicytunaavocadowraps
✅ Completato: spicytunaavocadowraps

[5/24] 🔄 Processing: dressedupmeatballs
✅ Completato: dressedupmeatballs

[6/24] 🔄 Processing: breakfastburritos
✅ Completato: breakfastburritos

[7/24] 🔄 Processing: cucumberraita
✅ Completato: cucumberraita

[8/24] 🔄 Processing: microwaveeggsandwich
✅ Completato: microwaveeggsandwich

[9/24] 🔄 Processing: panfriedtofu
✅ Completato: panfriedtofu

[10/24] 🔄 Processing: broccolistirfry
✅ Completato: broccolistirfry

[11/24] 🔄 Processing: sautedmushrooms
✅ Completato: sautedmushrooms

[12/24] 🔄 Processing: microwavemugpizza

In [ ]:
import json
from pathlib import Path
from tqdm import tqdm

# ==========================
# CONFIGURAZIONE
# ==========================
INPUT_DIR = Path("/content/drive/MyDrive/MistakeDetection/task_graphs")
OUTPUT_DIR = Path("/content/drive/MyDrive/MistakeDetection/task_graphs_fixed")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ==========================
# FUNZIONE DI FIX
# ==========================
def fix_recipe_json(recipe: dict):
    steps = recipe.get("steps", {})
    edges = recipe.get("edges", [])

    # Ordina gli step per indice numerico
    old_ids = sorted([int(k) for k in steps.keys()])

    # Mapping vecchio_id -> nuovo_id
    id_mapping = {old_id: new_id for new_id, old_id in enumerate(old_ids)}

    # Se non ci sono buchi, non serve fare nulla
    if old_ids == list(range(len(old_ids))):
        return recipe, False

    # Ricostruisci steps
    new_steps = {
        str(id_mapping[int(k)]): v
        for k, v in steps.items()
    }

    # Ricostruisci edges
    new_edges = []
    for src, tgt in edges:
        if src in id_mapping and tgt in id_mapping:
            new_edges.append([
                id_mapping[src],
                id_mapping[tgt]
            ])
        else:
            # edge inconsistente (raro ma possibile)
            print(f"⚠️ Edge ignorato: {src} -> {tgt}")

    fixed_recipe = {
        "steps": dict(sorted(new_steps.items(), key=lambda x: int(x[0]))),
        "edges": new_edges
    }

    return fixed_recipe, True


# ==========================
# LOOP SU TUTTI I JSON
# ==========================
fixed_count = 0
total_count = 0

for json_path in tqdm(list(INPUT_DIR.glob("*.json"))):
    total_count += 1

    with open(json_path, "r", encoding="utf-8") as f:
        recipe = json.load(f)

    fixed_recipe, was_fixed = fix_recipe_json(recipe)

    if was_fixed:
        fixed_count += 1

    out_path = OUTPUT_DIR / json_path.name
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(fixed_recipe, f, indent=2, ensure_ascii=False)

print("\n==============================")
print(f"File totali analizzati: {total_count}")
print(f"File corretti: {fixed_count}")
print(f"Output salvato in: {OUTPUT_DIR}")
print("==============================")


100%|██████████| 24/24 [00:01<00:00, 14.73it/s]


File totali analizzati: 24
File corretti: 3
Output salvato in: /content/drive/MyDrive/MistakeDetection/task_graphs_fixed


In [ ]:
import torch

data = torch.load('/content/drive/MyDrive/MistakeDetection/recipe_text_step_embeddings/buttercorncup.pt', weights_only=False)
embeddings = data['embeddings']
metadata = data['metadata']

In [ ]:
embeddings.shape

torch.Size([14, 768])

In [ ]:
metadata

{'edges': [[8, 1],
  [4, 1],
  [7, 3],
  [11, 5],
  [10, 6],
  [1, 6],
  [9, 8],
  [3, 9],
  [12, 9],
  [8, 10],
  [2, 11],
  [0, 2],
  [0, 4],
  [5, 7],
  [7, 12],
  [6, 13]],
 'step_ids': ['0',
  '1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '11',
  '12',
  '13'],
 'original_steps': {'0': 'START',
  '1': 'add-add lime juice to the bowl',
  '2': 'Measure-Measure 2 cups of frozen corn',
  '3': 'Add-1 teaspoon of pepper powder to the bowl',
  '4': 'Extract-Extract lime juice from 1/3 lime',
  '5': 'Add-Add the corn into a microwave-safe bowl',
  '6': 'Mix-Mix the contents of the bowl well',
  '7': 'Microwave-Microwave the corn for 2 minutes',
  '8': 'Microwave-Microwave the corn for 3 more minutes',
  '9': 'stir-then stir the bowl',
  '10': 'Add-Add 1 teaspoon salt to the bowl',
  '11': 'Thaw-Thaw the frozen corn by putting it in a sieve and running it under cold water',
  '12': 'Add-Add 1 teaspoon of softened butter',
  '13': 'END'},
 'texts': ['START',
  'add-

# Video clustering extraction

In [4]:
import json
import os
import glob

# Carica le informazioni sugli step delle ricette
recipe_steps_path = "/content/drive/MyDrive/MistakeDetection/annotation_json/recipe_steps_analysis.json"
with open(recipe_steps_path, 'r') as f:
    recipe_steps_data = json.load(f)

# Directory dei file video
video_dir = "/content/data/egovlp/egovlp/"
video_files = glob.glob(os.path.join(video_dir, "*.npz"))

print(f"📊 Trovati {len(video_files)} file video da processare")

# Costruisci la lista di input con il formato: video_path:num_clusters
input_list = []
skipped_count = 0

for video_file in video_files:
    filename = os.path.basename(video_file)

    # Estrai l'ID della ricetta (primo numero nel nome del file)
    # Es: "1_10_360p.mp4_1s_1s.npz" -> recipe_id = "1"
    recipe_id = filename.split('_')[0]

    # Ottieni max_steps per questa ricetta
    if recipe_id in recipe_steps_data:
        max_steps = recipe_steps_data[recipe_id]['max_steps']
        num_clusters = max_steps + 1

        # Aggiungi alla lista nel formato video_path:num_clusters
        input_list.append(f"{video_file}:{num_clusters}")
    else:
        print(f"⚠️ Recipe ID {recipe_id} non trovato in recipe_steps_analysis.json, saltando {filename}")
        skipped_count += 1

print(f"\n📋 Video da processare: {len(input_list)}")
print(f"⏭️ Video saltati: {skipped_count}")

if len(input_list) > 0:
    # Costruisci la stringa di input separando con spazi
    input_string = " ".join(input_list)

    print(f"\n🚀 Esecuzione batch di {len(input_list)} video...")
    print(f"   Primo video: {os.path.basename(input_list[0])}")
    print(f"   Ultimo video: {os.path.basename(input_list[-1])}")

    # Cambia directory
    %cd /content/HiERO-main

    # Esegui infer.py con tutti i file in un unico comando
    !python infer.py \
      --checkpoint /content/HiERO-main/checkpoints/hiero_egovlp.pth \
      --input {input_string} \
      --output /content/outputs/embeddings \
      --device cuda

    print("\n✅ Processamento batch completato!")
else:
    print("\n❌ Nessun video da processare!")

📊 Trovati 384 file video da processare

📋 Video da processare: 384
⏭️ Video saltati: 0

🚀 Esecuzione batch di 384 video...
   Primo video: 7_1_360p_224.mp4_1s_1s.npz:12
   Ultimo video: 13_24_360p_224.mp4_1s_1s.npz:13
/content
tokenizer_config.json: 100% 48.0/48.0 [00:00<00:00, 283kB/s]
config.json: 100% 483/483 [00:00<00:00, 4.28MB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 3.78MB/s]
tokenizer.json: 100% 466k/466k [00:00<00:00, 33.3MB/s]
config.json: 100% 483/483 [00:00<00:00, 4.61MB/s]
2025-12-22 17:54:39.068323: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766426079.319713    2124 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766426079.387034    2124 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory

In [5]:
# @title 🛠️ Fix: Recupero Cluster (numero variabile) dai file singoli
import numpy as np
import glob
import os
from tqdm import tqdm

OUTPUT_DIR = "/content/outputs/embeddings"
NEW_ALL_EMB_PATH = os.path.join(OUTPUT_DIR, "all_embeddings_clusters.npz")

print(f"📂 Cerco file singoli in: {OUTPUT_DIR}")
npz_files = glob.glob(os.path.join(OUTPUT_DIR, "*_embeddings.npz"))
print(f"   Trovati {len(npz_files)} file.")

clusters_dict = {}
count_ok = 0

if len(npz_files) > 0:
    print("\n🔄 Estrazione chiave 'clusters' (numero variabile di steps)...")

    for fpath in tqdm(npz_files):
        try:
            filename = os.path.basename(fpath)
            # Il nome file è tipo "video_name.mp4_1s_1s_embeddings.npz"
            # Vogliamo la chiave pulita: "video_name.mp4_1s_1s"
            video_key = filename.replace("_embeddings.npz", "")

            data = np.load(fpath)

            # Cerchiamo varianti del nome chiave per sicurezza
            keys_in_file = list(data.files)
            target_key = None
            if 'clusters' in keys_in_file:
                target_key = 'clusters'
            elif 'embeddings' in keys_in_file:
                target_key = 'embeddings'

            if target_key:
                arr = data[target_key]
                # Accettiamo qualsiasi numero di cluster (non più fisso a 8)
                if len(arr.shape) == 2 and arr.shape[1] == 768:
                    clusters_dict[video_key] = arr
                    count_ok += 1
                else:
                    print(f"⚠️ Shape inaspettato per {video_key}: {arr.shape}")

        except Exception as e:
            print(f"❌ Errore lettura {filename}: {e}")

    print(f"\n✅ Recuperati {count_ok} video con cluster.")

    if count_ok > 0:
        # Salviamo il nuovo file aggregato
        np.savez_compressed(NEW_ALL_EMB_PATH, **clusters_dict)
        print(f"💾 Salvato nuovo file aggregato: {NEW_ALL_EMB_PATH}")

        # Analisi rapida del nuovo file
        sample_key = list(clusters_dict.keys())[0]
        sample_arr = clusters_dict[sample_key]
        print(f"\n🔬 Esempio '{sample_key}':")
        print(f"   Shape: {sample_arr.shape}")
        print(f"   Numero di cluster: {sample_arr.shape[0]}")
        print(f"   Dimensione embedding: {sample_arr.shape[1]}")
    else:
        print("⚠️ Nessun cluster trovato nei file singoli. Controlla i log di inferenza.")

else:
    print("❌ Nessun file .npz trovato nella cartella.")

📂 Cerco file singoli in: /content/outputs/embeddings
   Trovati 385 file.

🔄 Estrazione chiave 'clusters' (numero variabile di steps)...


100%|██████████| 385/385 [00:01<00:00, 239.46it/s]



✅ Recuperati 384 video con cluster.
💾 Salvato nuovo file aggregato: /content/outputs/embeddings/all_embeddings_clusters.npz

🔬 Esempio '16_39_360p_224.mp4_1s_1s':
   Shape: (25, 768)
   Numero di cluster: 25
   Dimensione embedding: 768


In [6]:
# @title 🔬 Analisi Finale: Verifica Struttura con Numero Variabile di Step
import numpy as np
import json
import os

# Carica le informazioni sugli step delle ricette
recipe_steps_path = "/content/drive/MyDrive/MistakeDetection/annotation_json/recipe_steps_analysis.json"
with open(recipe_steps_path, 'r') as f:
    recipe_steps_data = json.load(f)

# Percorso del file clusters
CLUSTERS_PATH = "/content/outputs/embeddings/all_embeddings_clusters.npz"

print(f"📂 Analisi rigorosa file: {CLUSTERS_PATH} ...\n")

if not os.path.exists(CLUSTERS_PATH):
    print("❌ File non trovato!")
else:
    try:
        data = np.load(CLUSTERS_PATH, allow_pickle=True)
        keys = list(data.keys())
        total_videos = len(keys)

        print(f"1️⃣  CONTEGGIO VIDEO:")
        print(f"    Trovati {total_videos} video unici (chiavi).")

        # --- VERIFICA CON NUMERO VARIABILE DI STEP ---
        print(f"\n2️⃣  VERIFICA DIMENSIONI (Numero variabile di step + 1)...")

        correct_count = 0
        error_count = 0
        shape_distribution = {}

        # Iteriamo su TUTTO il dataset
        for key in keys:
            arr = data[key]

            # Estrai recipe_id dal nome del video
            recipe_id = key.split('_')[0]

            # Traccia distribuzione delle shape
            shape_str = f"{arr.shape[0]}x{arr.shape[1]}"
            shape_distribution[shape_str] = shape_distribution.get(shape_str, 0) + 1

            # Verifica se la ricetta esiste nel nostro database
            if recipe_id in recipe_steps_data:
                expected_clusters = recipe_steps_data[recipe_id]['max_steps'] + 1
                expected_shape = (expected_clusters, 768)

                if arr.shape == expected_shape:
                    correct_count += 1
                else:
                    error_count += 1
                    # Stampiamo l'errore solo per i primi 5 casi
                    if error_count <= 5:
                        print(f"    ⚠️ Errore su '{key}' (Recipe {recipe_id}):")
                        print(f"       Atteso: {expected_shape}, Trovato: {arr.shape}")
            else:
                error_count += 1
                if error_count <= 5:
                    print(f"    ⚠️ Recipe ID {recipe_id} non trovato per video '{key}'")

        # --- REPORT FINALE ---
        print(f"\n📊 REPORT STATISTICO:")
        print(f"    ✅ Video corretti: {correct_count} su {total_videos}")
        print(f"    ❌ Video con errori: {error_count}")

        print(f"\n📈 DISTRIBUZIONE SHAPE:")
        for shape, count in sorted(shape_distribution.items()):
            print(f"    {shape}: {count} video")

        print(f"\n💡 CONCLUSIONE:")
        if error_count == 0 and total_videos > 0:
            print("    🎉 SUCCESSO TOTALE!")
            print("       Tutti i video hanno il numero corretto di cluster")
            print("       basato sul max_steps della loro ricetta.")
        else:
            print(f"    ⚠️ Ci sono {error_count} video con anomalie. Controlla gli errori sopra.")

    except Exception as e:
        print(f"❌ Errore critico lettura file: {e}")

📂 Analisi rigorosa file: /content/outputs/embeddings/all_embeddings_clusters.npz ...

1️⃣  CONTEGGIO VIDEO:
    Trovati 384 video unici (chiavi).

2️⃣  VERIFICA DIMENSIONI (Numero variabile di step + 1)...

📊 REPORT STATISTICO:
    ✅ Video corretti: 384 su 384
    ❌ Video con errori: 0

📈 DISTRIBUZIONE SHAPE:
    10x768: 18 video
    12x768: 63 video
    13x768: 52 video
    14x768: 15 video
    16x768: 19 video
    17x768: 45 video
    18x768: 17 video
    19x768: 48 video
    20x768: 30 video
    21x768: 29 video
    25x768: 16 video
    27x768: 16 video
    8x768: 16 video

💡 CONCLUSIONE:
    🎉 SUCCESSO TOTALE!
       Tutti i video hanno il numero corretto di cluster
       basato sul max_steps della loro ricetta.


In [7]:
# @title 5. 💾 Salvataggio Finale su Drive (File Unico + ZIP Backup con Progress Bar)
import os
import shutil
import zipfile
from google.colab import drive
from tqdm import tqdm

# Funzione per convertire byte in MB/GB
def format_bytes(size):
    power = 2**10
    n = 0
    power_labels = {0 : '', 1: 'KB', 2: 'MB', 3: 'GB', 4: 'TB'}
    while size > power:
        size /= power
        n += 1
    return f"{size:.2f} {power_labels[n]}"

# Funzione per calcolare dimensione cartella
def get_dir_size(path):
    total = 0
    count = 0
    for entry in os.scandir(path):
        if entry.is_file():
            total += entry.stat().st_size
            count += 1
    return total, count

# 1. Mount Drive
if not os.path.exists('/content/drive'):
    print("📂 Montaggio Google Drive...")
    drive.mount('/content/drive')

# --- CONFIGURAZIONE ---
SOURCE_DIR = "/content/outputs/embeddings"
DEST_DIR = "/content/drive/MyDrive/MistakeDetection"

# File specifico da salvare separatamente
SOURCE_FILE_SINGLE = os.path.join(SOURCE_DIR, "all_embeddings_clusters.npz")
DEST_NAME_SINGLE = "hiero_all_video_steps_max_steps_plus_1.npz"
DEST_PATH_SINGLE = os.path.join(DEST_DIR, DEST_NAME_SINGLE)

# Nome per lo ZIP
ZIP_NAME = "hiero_video_steps.zip"
ZIP_PATH_LOCAL = os.path.join("/content", ZIP_NAME) # Salviamo prima in locale
ZIP_PATH_DRIVE = os.path.join(DEST_DIR, ZIP_NAME)
# ----------------------

if os.path.exists(SOURCE_DIR) and os.listdir(SOURCE_DIR):
    print("📊 ANALISI DIMENSIONI IN CORSO...")

    # Calcolo pesi
    single_size = os.path.getsize(SOURCE_FILE_SINGLE) if os.path.exists(SOURCE_FILE_SINGLE) else 0
    dir_size, file_count = get_dir_size(SOURCE_DIR)

    print(f"   🔹 File Unico (Clusters): {format_bytes(single_size)}")
    print(f"   🔹 Cartella Completa ({file_count} file): {format_bytes(dir_size)}")

    # Controllo Spazio Disco Colab
    total_disk, used_disk, free_disk = shutil.disk_usage("/")
    print(f"   💾 Spazio Disco Colab Libero: {format_bytes(free_disk)}")

    # Warning se lo spazio è stretto (consideriamo che lo zip occupa spazio + overhead)
    if free_disk < dir_size * 1.1:
        print("\n⚠️ ATTENZIONE CRITICA: Potrebbe non esserci abbastanza spazio su Colab per creare lo zip!")
        print("   Consiglio: Salva solo il file unico (A) e salta lo ZIP.")

    print("\n🚀 INIZIO OPERAZIONI...")

    # A) SALVATAGGIO FILE UNICO
    if os.path.exists(SOURCE_FILE_SINGLE):
        print(f"\n📄 [1/2] Copia file unico su Drive...")
        try:
            shutil.copy(SOURCE_FILE_SINGLE, DEST_PATH_SINGLE)
            print(f"   ✅ Salvato: {DEST_NAME_SINGLE} ({format_bytes(single_size)})")
        except Exception as e:
            print(f"   ❌ Errore copia: {e}")

    # B) CREAZIONE ZIP (MANUALE CON PROGRESS BAR)
    # Usiamo zipfile invece di shutil.make_archive per mostrare il progresso ed evitare timeout "ciechi"
    print(f"\n🗜️ [2/2] Creazione ZIP di backup ({format_bytes(dir_size)})...")
    try:
        file_list = [f for f in os.listdir(SOURCE_DIR) if os.path.isfile(os.path.join(SOURCE_DIR, f))]

        with zipfile.ZipFile(ZIP_PATH_LOCAL, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for f in tqdm(file_list, desc="Comprimendo", unit="file"):
                file_path = os.path.join(SOURCE_DIR, f)
                zipf.write(file_path, f) # Aggiunge al zip col nome 'f'

        zip_size = os.path.getsize(ZIP_PATH_LOCAL)
        print(f"   📦 Zip creato in locale: {format_bytes(zip_size)}")

        print(f"   ☁️ Caricamento su Drive in corso...")
        shutil.copy(ZIP_PATH_LOCAL, ZIP_PATH_DRIVE)
        print("   ✅ ZIP salvato su Drive!")

        # Pulizia spazio locale
        os.remove(ZIP_PATH_LOCAL)
        print("   🧹 File temporaneo locale rimosso.")

    except Exception as e:
        print(f"   ❌ Errore ZIP: {e}")

    print("\n🎉 PROCEDURA COMPLETATA!")

else:
    print("❌ Cartella output vuota.")

📊 ANALISI DIMENSIONI IN CORSO...
   🔹 File Unico (Clusters): 17.15 MB
   🔹 Cartella Completa (386 file): 8.71 GB
   💾 Spazio Disco Colab Libero: 55.61 GB

🚀 INIZIO OPERAZIONI...

📄 [1/2] Copia file unico su Drive...
   ✅ Salvato: hiero_all_video_steps_max_steps_plus_1.npz (17.15 MB)

🗜️ [2/2] Creazione ZIP di backup (8.71 GB)...


Comprimendo:   5%|▌         | 21/386 [00:31<09:05,  1.49s/file]


KeyboardInterrupt: 